In [18]:
#Instalação
%pip install yfinance==0.2.41
%pip install crewai==0.28.8
%pip install 'crewai[tools]'
%pip install langchain==0.1.20
%pip install langchain-openai==0.1.7
%pip install langchain-community==0.0.38
%pip install duckduckgo-search==5.3.0
%pip install streamlit


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python3.11 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python3.11 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python3.11 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python3.11 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python3.11 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 24.0 -> 24

In [20]:
# LIBS IMPORTS
import json
import os
from datetime import datetime

import yfinance as yf

from crewai import Agent, Task, Crew, Process

from langchain.tools import Tool
from langchain_community.tools import DuckDuckGoSearchResults
from langchain_openai import ChatOpenAI
import streamlit as st

In [4]:
# CREATES THE YAHOO FINANCE TOOL
def fetch_stock_price(ticket):
  stock_prices = yf.download(ticket, start="2023-08-20", end="2024-08-20")
  return stock_prices

yahoo_finance_tool = Tool(
  name = "Yahoo finance tool",
  description = "Fetches stock prices for {ticket} from the last year about a specific company from Yahoo Finance API",
  func = lambda ticket: fetch_stock_price(ticket)
)

In [5]:
# IMPORTS OPEN_AI LLM - GPT
os.environ['OPENAI_API_KEY'] = st.secrets['OPEN_API_KEY']
llm = ChatOpenAI(model="gpt-3.5-turbo")

In [6]:
# CREATES THE AGENT FOR STOCK PRICE ANALYSES
stockPriceAnalyst = Agent(
  role = "Senior Stock Price Analyst",
  goal = "Find the {ticket} stock price and analyses trends",
  backstory = "You're a highly experienced in analyzing the price of an specific stock and make predictions about its future price",
  verbose = True,
  llm = llm,
  max_iter = 5,
  memory = True,
  tools = [yahoo_finance_tool],
  allow_delegation = False
)

In [7]:
# CREATES THE TASK FOR GET STOCK PRICE THAT WILL BE ANALYZED BY STOCK PRICE AGENT
getStockPrice = Task(
  description = "Analyze the stock {ticket} price history and create a trend analyses of up, down or sideways",
  expected_output = """Specify the current trend stock price - up, down or sideways.
  eg. stock = "AAPL, price UP."
""",
  agent = stockPriceAnalyst
)

In [8]:
# IMPORTS THE NEWS TOOL
search_tool = DuckDuckGoSearchResults(backend="news", num_results=10)

In [9]:
newsAnalyst = Agent(
  role = "Stock News Analyst",
  goal = """Create a short summary of the market news related to the stock {ticket} company. 
  Specify the current trend - up, down or sideways with the news context. For each request stock asset, specify
  a number between 0 and 100, where 0 is extreme fear and 100 is extreme greed.""",
  backstory = """You're highly experienced in analyzing the market trends and news and have tracked assest
  for more than 10 years.
  You're also master level analyts in the tradicional markets and have deep understanding of 
  human psychology.
  You understand news, their titles and information, but you look at those with a health dose of skepticism.
  You also consider the source of the news articles.
  """,
  verbose = True,
  llm = llm,
  max_iter = 10,
  memory = True,
  tools = [search_tool],
  allow_delegation = False
)

In [10]:
# CREATES THE NEWS TASK
getNews = Task(
  description = f"""Take the stock and always include BTC to it (if not included).
  Use the search tool to search each one individually.
  The current date is {datetime.now()}.
  Compose the results into a helpfull report""",
  expected_output = """A summary of the overall market and one sen  tence summary for each request asset.
  Include a fear/greed score for each asset based on the news. Use format:
  <STOCK ASSET>
  <SUMMARY BASED ON NEWS>
  <TREND PREDICTION>
  <FEAR/GREED SCORE>
  """,
  agent = newsAnalyst
)

In [11]:
stockAnalystWrite = Agent(
  role = "Senior Stock Analyts Writer",
  goal = """Analyze the trends price and news and write a insighfull compelling and informative 3 paragraph 
  long newsletter based on the stock report and price trend.""",
  backstory = """You're widely accepted as the best stock analyst in the market. You understand complex
  concepts and create compelling stories and narratives that resonate with wider audiences.

  You understand macro factors and combine multiples theories - e.g cycle theory and fundamental analyses.
  You're able to hold multiple opinions when analyzing anything.
""",
verbose = True,
llm = llm,
max_iter = 5,
memory = True,
allow_delegation = True
)

In [12]:
writeAnalyses = Task(
  description = """Use the stock price trend and the stock news report to create analyses and write 
  the newsletter about the {ticket} company that is brief and highlights the most important points.
  Focus on the stock price trend, news and fear/greed score. What are near future considerations?
  Include the previous analyses of stock trend and news summary.
  """,
  expected_output = """An eloquent 3 paragraphs newsletter formated as markdown in an easy readable 
  manner. It should contain:
  - 3 bullets executive summary
  - Introdution - set the overall picture and spike up the interest
  - main part provides the meat of the analysis including the news summary and fear/greed scores
  - summary - key facts and concrete future trend prediction - up, down or sideways.
  """,
  agent = stockAnalystWrite,
  context = [getStockPrice, getNews]
)

In [13]:
crew = Crew(
  agents = [stockPriceAnalyst, newsAnalyst, stockAnalystWrite],
  tasks = [getStockPrice, getNews, writeAnalyses],
  verbose = True,
  process = Process.hierarchical,
  full_output = True,
  share_crew = False,
  manager_llm = llm,
  max_iter = 15
)

In [14]:
result = crew.kickoff(inputs = {'ticket': 'AAPL'})

 [DEBUG]: Working Agent: Crew Manager
 [INFO]: Starting Task: Analyze the stock AAPL price history and create a trend analyses of up, down or sideways


> Entering new CrewAgentExecutor chain...
I need to gather information from my team in order to analyze the stock AAPL price history and determine the current trend.

Action: Delegate work to co-worker
Action Input: {"coworker": "Senior Stock Price Analyst", "task": "Analyze AAPL stock price history and identify trend", "context": "The task is to analyze the historical stock price data of AAPL and determine if the trend is up, down, or sideways. Please use all available data and tools to provide an accurate analysis."}

> Entering new CrewAgentExecutor chain...
I need to gather the historical stock price data of AAPL and analyze it to determine the trend.

Action: Yahoo finance tool
Action Input: {"ticket": "AAPL"}

[*********************100%%**********************]  1 of 1 completed


 

                  Open        High         Low       Close   Adj Close  \
Date                                                                     
2023-08-21  175.070007  176.130005  173.740005  175.839996  174.944778   
2023-08-22  177.059998  177.679993  176.250000  177.229996  176.327698   
2023-08-23  178.520004  181.550003  178.330002  181.119995  180.197906   
2023-08-24  180.669998  181.100006  176.009995  176.380005  175.482025   
2023-08-25  177.380005  179.149994  175.820007  178.610001  177.700684   
...                ...         ...         ...         ...         ...   
2024-08-13  219.009995  221.889999  219.009995  221.270004  221.270004   
2024-08-14  220.570007  223.029999  219.699997  221.720001  221.720001   
2024-08-15  224.600006  225.350006  222.759995  224.720001  224.720001   
2024-08-16  223.919998  226.830002  223.649994  226.050003  226.050003   
2024-08-19  225.720001  225.990005  223.039993  225.889999  225.889999   

              Volume  
Date       

In [15]:
print(result)

{'final_output': "### AAPL Company Newsletter\n\n**Executive Summary:**\n- Positive sentiment among investors with significant upside ahead for AAPL.\n- Anticipation and excitement for Apple's upcoming new iPhone event in September.\n- Contrasting reports on Apple's innovation and stock performance.\n\n**Introduction:**\nApple Inc. (AAPL) continues to capture the attention of investors and analysts alike with its recent stock price trend pointing upwards. The company's resilience in the market and brand loyalty have positioned it as a strong investment choice, despite some contrasting views on its innovation and performance.\n\n**Main Analysis:**\nRecent news updates surrounding AAPL have been a mix of positive and cautionary signals. Billionaire Dan Loeb's optimism about Apple's future growth potential has fueled positive sentiment, while Warren Buffett's significant stock sale has raised questions about the company's trajectory. Analysts are eagerly awaiting Apple's new iPhone event 

In [24]:
list(result.keys())

['final_output', 'tasks_outputs']

In [29]:
print(result['final_output'])

### AAPL Company Newsletter

**Executive Summary:**
- Positive sentiment among investors with significant upside ahead for AAPL.
- Anticipation and excitement for Apple's upcoming new iPhone event in September.
- Contrasting reports on Apple's innovation and stock performance.

**Introduction:**
Apple Inc. (AAPL) continues to capture the attention of investors and analysts alike with its recent stock price trend pointing upwards. The company's resilience in the market and brand loyalty have positioned it as a strong investment choice, despite some contrasting views on its innovation and performance.

**Main Analysis:**
Recent news updates surrounding AAPL have been a mix of positive and cautionary signals. Billionaire Dan Loeb's optimism about Apple's future growth potential has fueled positive sentiment, while Warren Buffett's significant stock sale has raised questions about the company's trajectory. Analysts are eagerly awaiting Apple's new iPhone event in September, indicating high

### AAPL Company Newsletter

**Executive Summary:**
- Positive sentiment among investors with significant upside ahead for AAPL.
- Anticipation and excitement for Apple's upcoming new iPhone event in September.
- Contrasting reports on Apple's innovation and stock performance.

**Introduction:**
Apple Inc. (AAPL) continues to capture the attention of investors and analysts alike with its recent stock price trend pointing upwards. The company's resilience in the market and brand loyalty have positioned it as a strong investment choice, despite some contrasting views on its innovation and performance.

**Main Analysis:**
Recent news updates surrounding AAPL have been a mix of positive and cautionary signals. Billionaire Dan Loeb's optimism about Apple's future growth potential has fueled positive sentiment, while Warren Buffett's significant stock sale has raised questions about the company's trajectory. Analysts are eagerly awaiting Apple's new iPhone event in September, indicating high expectations for the tech giant. On the other hand, concerns about innovation and stock performance have prompted discussions about the company's long-term outlook.

**Summary and Future Prediction:**
In conclusion, Apple's stock trend remains positive, supported by a fear/greed score of 80, reflecting high levels of investor confidence. Despite some fluctuations, the overall sentiment is optimistic for AAPL's future prospects. The company's ability to innovate and adapt to market trends will be crucial in determining its future trajectory - a path that currently points towards continued growth and stability.
